In [1]:
%load_ext bigdata
%hive_start
%timeout 300

In [2]:
!hadoop fs -rm -f -r tmp;

Deleted tmp


In [3]:
%%hive
DROP TABLE IF EXISTS t0;
CREATE TABLE t0 (
    c1 STRING,
    c2 ARRAY<CHAR(1)>, 
    c3 MAP<STRING, INT>
    )
    ROW FORMAT DELIMITED 
        FIELDS TERMINATED BY '\t'
        COLLECTION ITEMS TERMINATED BY ','
        MAP KEYS TERMINATED BY '#'
        LINES TERMINATED BY '\n';

DROP TABLE IF EXISTS t0;
OK
Time taken: 9.821 seconds
CREATE TABLE t0 (
    c1 STRING,
    c2 ARRAY<CHAR(1)>, 
    c3 MAP<STRING, INT>
    )
    ROW FORMAT DELIMITED 
        FIELDS TERMINATED BY '\t'
        COLLECTION ITEMS TERMINATED BY ','
        MAP KEYS TERMINATED BY '#'
        LINES TERMINATED BY '\n';
OK
Time taken: 1.433 seconds


In [4]:
%%hive
LOAD DATA LOCAL INPATH 'data.tsv' INTO TABLE t0;

LOAD DATA LOCAL INPATH 'data.tsv' INTO TABLE t0;
Loading data to table default.t0
OK
Time taken: 1.869 seconds


In [8]:
%%hive
DROP TABLE IF EXISTS salida;
CREATE TABLE salida
AS
    SELECT letra, COUNT(c1)
    FROM
        t0 LATERAL VIEW explode(c3) t0 AS letra, valor
GROUP BY
    letra
ORDER BY
    letra;

DROP TABLE IF EXISTS salida;
OK
Time taken: 0.011 seconds
CREATE TABLE salida
AS
    SELECT letra, COUNT(c1)
    FROM
        t0 LATERAL VIEW explode(c3) t0 AS letra, valor
GROUP BY
    letra
ORDER BY
    letra;
Query ID = root_20200120003221_1d2fd978-598e-4c6f-9ac3-767e3042a1e1
Total jobs = 2
Launching Job 1 out of 2
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1579469532371_0011, Tracking URL = http://92c98434d4d4:8088/proxy/application_1579469532371_0011/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1579469532371_0011
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
2020-01-20 00:32:31,474 Stage-1 map = 0%,  redu

In [9]:
%%hive
--SELECT * 
--FROM salida;    

SELECT * 
FROM salida;    
OK
aaa	13
bbb	16
ccc	23
ddd	23
eee	15
fff	20
ggg	13
hhh	16
iii	18
jjj	18
Time taken: 2.523 seconds, Fetched: 10 row(s)


In [10]:
%%hive
INSERT OVERWRITE DIRECTORY 'tmp/000000_0'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
SELECT
    *
FROM
    salida;

INSERT OVERWRITE DIRECTORY 'tmp/000000_0'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
SELECT
    *
FROM
    salida;
Query ID = root_20200120003435_dc7fbe7d-95b4-48be-8d1f-fbc3eff0b9e0
Total jobs = 3
Launching Job 1 out of 3
Number of reduce tasks is set to 0 since there's no reduce operator
Starting Job = job_1579469532371_0013, Tracking URL = http://92c98434d4d4:8088/proxy/application_1579469532371_0013/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1579469532371_0013
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 0
2020-01-20 00:34:42,927 Stage-1 map = 0%,  reduce = 0%
2020-01-20 00:34:47,415 Stage-1 map = 100%,  reduce = 0%, Cumulative CPU 1.22 sec
MapReduce Total cumulative CPU time: 1 seconds 220 msec
Ended Job = job_1579469532371_0013
Stage-3 is selected by condition resolver.
Stage-2 is filtered out by condition resolver.
Stage-4 is filtered out by condition resolver.
Moving data to directory hdfs://0.0.0.0:9000/user/root/tmp/000000_

In [11]:
!hadoop fs -copyToLocal ./tmp/000000_0 output

In [12]:
%hive_quit